# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて新たに追加されたキーワードの定義を取得することを目的としています。具体的な目的は、取得した定義を用いて以下のことを行うことです：

1. トレーニング目的のために、既知の回答を持つ良質な質問を生成する。
2. `things`というキーワードに関する理解を深め、より多くの生成を促進する。
3. `answerer`という役割を持つエージェントがより正確に応答できるように情報を提供する。

ノートブックでは、`dictionaryapi`というAPIを使用して、キーワードの定義を取得しています。最終的な結果は`things_definitions.json`ファイルに保存されます。

使用されている主なライブラリおよびツールは以下の通りです：

- **Pandas**: データ解析のために利用され、キーワード情報をDataFrameとして管理しています。
- **Requests**: HTTPリクエストを送信するために使用し、APIから定義を取得します。
- **JSON**: JSONデータを扱うために使用し、APIからのレスポンスを処理します。
- **jq**: JSONデータを操作するために使用され、特定のフィールド（例: "meanings"）を抽出するために利用されています。
- **random, time**: タイミングやランダム性を制御するために使用されます。

ノートブックの工程としては以下が含まれます：

1. キーワードを含むCSVファイルを読み込み、PandasのDataFrameとして保存。
2. APIからのテストリクエストにより、特定の単語の定義を取得。
3. "things"カテゴリーのキーワードをフィルタリング。
4. 各キーワードに対して、APIを利用し、意味を取得する関数を生成。
5. 取得した定義をJSONファイルに保存。

このようにして、定義を効果的に収集し、モデルのトレーニングやエージェントの精度向上を目指しています。

---


# 用語概説 
以下に、初心者がつまずきやすい専門用語の簡単な解説を示します。特にマイナーなものや、実務を経験していないと馴染みが薄い用語に焦点を当てています。

1. **ドメイン知識**: 特定の分野や業界に関する深い知識のこと。このノートブックでは、「20の質問ゲーム」やその関連語の理解が求められます。

2. **API（Application Programming Interface）**: ソフトウェア同士が相互にやり取りするためのルールやインターフェース。ここでは辞書APIを使用して単語の定義を取得しています。

3. **HTTPリクエスト**: ウェブサーバーにデータを要求するためのメッセージ。`requests.get()`で行う操作がこれに該当します。

4. **JSON（JavaScript Object Notation）**: データを構造化するための軽量フォーマット。データの交換に広く使われ、APIの応答形式としても一般的です。

5. **jq**: JSONデータを操作するための軽量なコマンドラインツールおよびライブラリ。ここではAPIから取得したJSONデータをフィルタリングするために使われています。

6. **curl**: データを送受信するためのコマンドラインツール。URLに対してHTTPリクエストを行うためのコマンドとして使用されています。

7. **enumerate**: リストやイテラブルオブジェクトを順番に処理し、そのインデックスも同時に取得できるPythonの組み込み関数。ここではキーワードのリストを処理する際に使用されています。

8. **meanings**: 辞書APIからのレスポンスの中で、単語の意味に関する情報を含むフィールド。特に複数の意味や定義が存在する単語の情報を管理します。

9. **エラーハンドリング**: プログラムの実行中に発生する可能性のあるエラーを捕捉し、適切に対処する技術。ここでは、API呼び出しの結果のエラーチェックを行っています。

10. **タイムアウト**: システムやプロセスが一定の時間内に応答をしない場合に、実行を中止すること。APIリクエストを行うときに、過剰な時間をかけないように制御します。

11. **スリープ関数（time.sleep）**: プログラムの実行を一時停止させるための関数。ここでは、連続して行うAPIリクエストの間にランダムな待機時間を入れて、サーバーへの負担を軽減するために使用されています。

12. **peek（プログラムでの使われ方）**: データに対する簡易的な参照や確認を行うこと。ここでは、取得したデータをすぐに表示する様子からその概念が伺えます。

これらの用語についての理解が深まることで、ノートブックの内容がよりスムーズに理解できるようになるでしょう。

---


このノートブックでは、`dictionaryapi`を使って、[コンペティション](https://www.kaggle.com/competitions/llm-20-questions)に新たに追加されたキーワードの定義を探します。  
目標は、これらの定義を利用して以下のことを行うことです：
1. トレーニング目的のために、既知の回答を持つ（良い）質問を生成すること
1. キーワードである`things`についてより良く理解し、より多くの生成が可能になるようにすること
1. `answerer`の役割を持つエージェントに、より正確に応答できるように情報を提供すること（RAG？）

結果は`things_definitions.json`に保存されます。  
注意：
- 一部の単語には複数の定義があります（例：`Plate`には多くの定義があります）
- 多くの二語以上の単語（例：`Air compressor`）は、dictionaryapiには定義がありません。一部の二語のキーワードには定義があります（例：`Contact lenses`）
- ほとんどの一語のキーワードには少なくとも一つの定義があります（例外：RV、IVなど）

In [ ]:
%%capture
# jqをインストールするためのコマンドです。
# jqはJSONデータを操作するための軽量で柔軟なコマンドラインツールです。
!pip install jq

In [ ]:
import pandas as pd  # データ解析を行うためのPandasライブラリをインポートします
import requests  # HTTPリクエストを送信するためのRequestsライブラリをインポートします
import json  # JSONデータを扱うためのjsonライブラリをインポートします
import time  # 時間関連の処理を行うためのtimeライブラリをインポートします
import jq  # JSONデータを操作するためのjqライブラリをインポートします
import random  # ランダムな数値を生成するためのrandomライブラリをインポートします

In [ ]:
# keywords.csvファイルを読み込み、PandasのDataFrameとして保存します。
# index_col=0を指定することで、CSVの最初の列をインデックスとして使用します。
keywords = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/keywords.csv", index_col=0)

# 読み込んだキーワードのDataFrameを表示します。
keywords

In [ ]:
# keywords.pyというスクリプトを実行します。
# このスクリプト内で定義されている内容が必要です。
%run /kaggle/input/llm-20-questions/llm_20_questions/keywords.py

# KEYWORDS_JSONからキーワードを含む古いセットを作成します。
# 各カテゴリーからキーワードを抽出し、セットとして保存します。
keywords_old: set[str] = {details["keyword"] for category in json.loads(KEYWORDS_JSON) for details in category["words"]}

# カテゴリーが"place"のキーワードを現在のDataFrameからセットとして取得します。
keywords_place = set(keywords.loc[keywords.category == "place", "keyword"])

# 現在の場所のキーワードの中で、以前のJSONに存在しないものを出力します。
print(f"Current place keyword not in the previous json: {keywords_place - keywords_old}")

# 古い場所のキーワードの中で、現在存在しないものを出力します。
print(f"Old place keyword not in current: {keywords_old - keywords_place}")

# 使用が終わったため、keywords_placeとkeywords_oldを削除します。
del keywords_place, keywords_old

# 事物

In [ ]:
# "things"カテゴリーのキーワードをフィルタリングして表示します。
# keywords DataFrameの中から、カテゴリーが"things"である行を取得します。
keywords[keywords.category == "things"]

In [ ]:
# APIのURLを定義します。ここでは、特定の単語の定義を取得するためのURLを作成します。
url_api = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"

# APIをテストするために、"Zinc"という単語の定義を取得します。
r = requests.get(url_api.format(word="Zinc"))

# リクエストのステータスコードを表示します。
# ステータスコードが200の場合は成功、他のコードはエラーを示します。
r.status_code

In [ ]:
# 取得したレスポンスのJSONデータを表示します。
# これにより、"Zinc"という単語の定義やその他の情報を確認できます。
r.json()

In [ ]:
# JSONデータをフィルタリングするための例としてjqを使用します。
# "meanings"フィールドを持つ要素を選択し、その内容を取得します。
# JSONの長さが0より大きい要素を選び、最初の"meanings"を抽出します。
jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()

## Bashでの代替手段
jqはコマンドラインで使用するJSONプロセッサです。

In [ ]:
# curlコマンドを使用して、"Wristband"という単語の定義をAPIから取得し、jqを使って出力をフィルタリングします。
# ここでは、取得したJSONデータから"meanings"を抽出します。
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Wristband}" | jq '.[]["meanings"]'

In [ ]:
# curlコマンドを使用して、"Plate"という単語の定義をAPIから取得し、
# 取得したJSONデータをそのまま表示します。
!curl "https://api.dictionaryapi.dev/api/v2/entries/en/{Plate}" | jq '.[]'

In [ ]:
# "things"カテゴリーに属するキーワードの中で、一番長いキーワードの文字数を取得します。
# keywords DataFrameから、カテゴリーが"things"のキーワードを選択し、それらの長さを計算して最大値を求めます。
keywords.loc[keywords.category == "things", "keyword"].str.len().max()

In [ ]:
# 前のバージョンからの定義を読み込みます。
# "things_definitions.json"ファイルを開き、内容をJSON形式で読み込みます。
with open("/kaggle/input/20questions-keywords-things-definitions/things_definitions.json", "r") as f:
    keywords_definitions = json.load(f)

# 全体の"things"キーワードの数を表示します。
print(len(keywords_definitions), "things keywords total")

# 定義が見つかったキーワードだけをフィルタリングします。
keywords_definitions = {k: v for k, v in keywords_definitions.items() if len(v) > 0}

# 定義が見つかったキーワードの数を表示します。
print(len(keywords_definitions), "with found definitions")

# 最初のキーワードとその定義を表示します。
next(iter(keywords_definitions.items()))

In [ ]:
%%time
# 定義生成器関数を定義します。この関数はキーワードの定義を取得し、成功した場合はその結果を返します。
def definition_generator():
    fail = 0  # 失敗したリクエストのカウント
    for num, keyword in enumerate(keywords.loc[keywords.category == "things", "keyword"]):
        r = requests.get(url_api.format(word=keyword))  # APIからキーワードの定義を取得します。
        if keyword in keywords_definitions:  # 既に定義がある場合
            yield keyword, keywords_definitions[keyword]  # 定義を返します。
            continue
        if r.status_code != 200:  # リクエストが成功しない場合
            print(f"Error {r.status_code=} {keyword = :30} failrate:{fail/num:.2%}")
            yield keyword, []  # 空のリストを返します。
            fail += 1  # 失敗カウントを増やします。
            continue
        try:
            # 成功した場合、意味を抽出して返します。
            yield keyword, jq.compile('select(length > 0) []["meanings"]').input_value(r.json()).first()
        except (ValueError, JSONDecodeError) as e:
            # エラーが発生した場合、エラーメッセージを表示します。
            print(f"{type(e).__name__} {e} parsing response {keyword = } failrate:{fail/num:.2%})")
            print(e)
            fail += 1  # 失敗カウントを増やします。
            yield keyword, []  # 空のリストを返します。
        time.sleep(1 + random.random() / 2)  # タイムアウトを避けるためにリクエストを遅くします。
    print(f"{fail=} ({fail/num:.2%})")  # 失敗率の表示

# キーワードとその定義を生成します。
things_definitions = {keyword: definitions for keyword, definitions in definition_generator()}

# 取得した定義の総数を表示します。
len(things_definitions)

# 結果
& 保存

In [ ]:
# 取得した定義を"things_definitions.json"ファイルに保存します。
with open("things_definitions.json", "w") as f:
    json.dump(things_definitions, f)

# 保存した定義を表示します。
things_definitions